In [2]:
!pip install tensorflow

from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/ml-datasets/resized_training"

import os
import cv2

print(len(os.listdir(path)), os.listdir(path))

raw_dataset = {}
for artist in os.listdir(path) :
  raw_dataset[artist] = []
  for image in os.listdir(os.path.join(path, artist)) :
    raw_dataset[artist].append(cv2.imread(os.path.join(path, artist, image)))
  print(artist, ":")
  print(raw_dataset[artist])

Streaming output truncated to the last 5000 lines.
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8), array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [2, 0, 1],
        [2, 0, 1],
        [2, 0, 1]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [2, 0, 1],
        [2, 0, 1],
        [2, 0, 1]]], dtype=uint8), array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
     